<a href="https://colab.research.google.com/github/G0nkly/pytorch_sandbox/blob/main/vit/SigLip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from typing import Optional
import torch
import torch.nn as nn

In [ ]:
class SiglipVisionConfig:

  def __init__(
      self,
      hidden_size=768,
      intermediate_size=3072,
      num_hidden_layers=12,
      num_attention_heads=12,
      num_channels=3,
      image_size=224,
      patch_size=16,
      layer_norm_eps=1e-6,
      attention_dropout=0.0,
      num_image_tokens: int = None,
      **kwargs
  ):
    super().__init__()

    self.hidden_size = hidden_size
    self.intermediate_size = intermediate_size
    self.num_hidden_layers = num_hidden_layers
    self.num_attention_heads = num_attention_heads
    self.num_channels = num_channels
    self.patch_size = patch_size
    self.image_size = image_size
    self.attention_dropout = attention_dropout
    self.layer_norm_eps = layer_norm_eps
    self.num_image_tokens = num_image_tokens

In [ ]:
class SiglipVisionEmbeddings(nn.Module):

  def __init__(self, config: SiglipVisionConfig):
    super().__init__()
    self.config = config
    self.embed_dim = config.hidden_size
    self.image_size = config.image_size
    self.patch_size = config.patch_size

    self.patch_embedding = nn.Conv2d(
        in_channels=config.num_channels,
        out_channels=self.embed_dim,
        kernel_size=self.patch_size,
        stride=self.patch_size,
        padding="valid" # this indicates that no padding is added
    )

    self.num_patches = (self.image_size // self.patch_size) ** 2
    self.num_positions = self.num_patches
    self.position_embedding = nn.Embedding(self.num_positions, self.embed_dim)
    self.register_buffer(
        "position_ids",
        torch.arange(self.num_positions).expand((1, -1))
        persistent=False
    )

  def forward(self, pixel_values: torch.FloatTensor) -> torch.Tensor:
    _, _, height, width = pixel_values.shape # (B, C, H, W)
    patch_embeds = self.patch_embedding(pixel_values) # (B, embed_dim, num_patches[num_patches_h * num_patches_w])
    embeddings = patch_embeds.flatten(2)
    embeddings = embeddings.transpose(1, 2) # (B, num_patches, embed_dim)
    embeddings = embeddings + self.position_embedding(self.position_ids)
    return embeddings # (B, num_patches, embed_dim)


In [ ]:
class SiglipVisionTransformer(nn.Module):

  def __init__(self, config: SiglipVisionConfig):
    super().__init__()
    self.config = config
    embed_dim = config.hidden_size
    self.embeddings = SiglipVisionEmbeddings(config)
    self.encoder = SiglipEncoder(config)
    self.post_layernorm = nn.LayerNorm(embedd_dim, eps=config.layer_norm_eps)

  def forward(self, pixel_values: torch.Tensor) -> torch.Tensor:
    # B, C, H, W -> B, num_patches, embed_dim
    hidden_states = self.embeddings(pixel_values)
    last_hidden_state = self.encoder(input_embed=hidden_states)
    last_hidden_state = self.post_layernorm(last_hidden_state)

    return last_hidden_state

class SiglipVisionModel(nn.Module):

  def __init__(self, config: SiglipVisionConfig):
    super().__init__()
    self.config = config
    self.vision_model = SiglipVisionTransformer(config)

  def forward(self, pixel_values) -> Tuple:
    # B, C, H, W -> B, num_patches, embed_dim
    return self.vision_model(pixel_values=pixel_values)

NameError: name 'Tuple' is not defined